# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum


# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1
spark.sql("SELECT DISTINCT page FROM user_log_table WHERE userId = ''").show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [6]:
# TODO: write your code to answer question 3
spark.sql("SELECT count(distinct userId) FROM user_log_table WHERE gender = 'F'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [9]:
# TODO: write your code to answer question 4
spark.sql("""
    SELECT artist, count(1) as c 
    FROM user_log_table 
    WHERE page = 'NextSong' 
    GROUP BY artist ORDER BY c desc 
    LIMIT 5""").show()

+--------------------+---+
|              artist|  c|
+--------------------+---+
|            Coldplay| 83|
|       Kings Of Leon| 69|
|Florence + The Ma...| 52|
|            BjÃÂ¶rk| 46|
|       Dwight Yoakam| 45|
+--------------------+---+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [39]:
# TODO: write your code to answer question 5
user_period = spark.sql("""
    SELECT ts, page, userId, SUM( case when page = 'Home' then 1 else 0 end) OVER (partition by userid order by ts) AS period 
    FROM user_log_table 
    WHERE (page = 'NextSong' OR page ='Home')
    ORDER BY ts """)

In [40]:
user_period.show()

+-------------+--------+------+------+
|           ts|    page|userId|period|
+-------------+--------+------+------+
|1513720872284|NextSong|  1046|     0|
|1513720878284|NextSong|  1000|     0|
|1513720881284|NextSong|  2219|     0|
|1513720905284|NextSong|  2373|     0|
|1513720913284|    Home|  1747|     1|
|1513720955284|NextSong|  1162|     0|
|1513720959284|NextSong|  1061|     0|
|1513720959284|    Home|   748|     1|
|1513720980284|    Home|   597|     1|
|1513720983284|NextSong|  1806|     0|
|1513720993284|NextSong|   748|     1|
|1513721031284|NextSong|  1176|     0|
|1513721045284|NextSong|  2164|     0|
|1513721058284|NextSong|  2146|     0|
|1513721077284|NextSong|  2219|     0|
|1513721088284|    Home|  1176|     1|
|1513721095284|NextSong|  2904|     0|
|1513721097284|NextSong|   597|     1|
|1513721104284|NextSong|  1046|     0|
|1513721104284|NextSong|   226|     0|
+-------------+--------+------+------+
only showing top 20 rows



In [41]:
user_period.createOrReplaceTempView("user_period_table")

In [45]:
user_period_total = spark.sql("""
    SELECT avg( total ) from (
        SELECT userid, period, count(1) AS total 
            FROM user_period_table 
            WHERE page = 'NextSong' 
            GROUP BY userid,period
    )
""")

In [46]:
user_period_total.show()

+------------------+
|        avg(total)|
+------------------+
|6.9558333333333335|
+------------------+

